# Text Chunking
This lab will walk you through various methods to perform chunking of your text. Retrieval is a very important step in RAG architecture. Semantic search requires you take your knowledge/text and convert that into embeddings and store them in a search engine that offers vector search capability. To convert your documents into embedding, you will need to split them into smaller pieces, popularly called "Chunks". This technique is known as "Chunking". Chunking is necessary because a large text passage may reduce its specificity, it may conflate different topics or concepts making it not best match for a query about a topic. This would mean even if there is a very relevant information in one part of the text passage, the similarity of text passage as a whole to user's query may be low, this may exclude the text passage from top semantic search results.


There is a [great resource by Greg Kamradt](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb) from where you can learn about various ways to chunk text  


In [ ]:
!pip install langchain langchain_community pypdf langchain_experimental --quiet
!pip install -qU langchain-text-splitters
!pip install --upgrade --quiet  boto3
!pip install pdfminer.six --quiet
!pip install amazon-textract-caller --quiet
!pip install amazon-textract-textractor --quiet
!pip install opensearchpy

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.chat_models import BedrockChat
from langchain.chains import ConversationalRetrievalChain

from langchain_community.embeddings import BedrockEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

import boto3
import os
import time
import json
import pandas as pd
from tqdm import tqdm
import sagemaker
from opensearchpy import OpenSearch, RequestsHttpConnection
from sagemaker import get_execution_role
import random 
import string
import s3fs
from urllib.parse import urlparse
from IPython.display import display, HTML
from alive_progress import alive_bar
from opensearch_py_ml.ml_commons import MLCommonClient
from requests_aws4auth import AWS4Auth
import requests 


In [ ]:
# Create a Boto3 session
session = boto3.Session()

# Get the account id
account_id = boto3.client('sts').get_caller_identity().get('Account')

# Get the current region
region = session.region_name

cfn = boto3.client('cloudformation')
bedrock_client = boto3.client('bedrock-runtime')

# Method to obtain output variables from Cloudformation stack. 
def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to for the rest of the demo
cloudformation_stack_name = "genai-data-foundation-workshop"

outputs = get_cfn_outputs(cloudformation_stack_name)
aos_host = outputs['OpenSearchDomainEndpoint']
s3_bucket = outputs['s3BucketTraining']
bedrock_inf_iam_role = outputs['BedrockBatchInferenceRole']
bedrock_inf_iam_role_arn = outputs['BedrockBatchInferenceRoleArn']
sagemaker_notebook_url = outputs['SageMakerNotebookURL']

# We will just print all the variables so you can easily copy if needed.
outputs

In [ ]:
import boto3
import os


def upload_file_to_s3(file_path, bucket_name, prefix):
    s3 = boto3.client('s3')
    file_name = os.path.basename(file_path)
    object_key = f"{prefix}/{file_name}" if prefix else file_name
    
    with open(file_path, 'rb') as file_data:
        s3.upload_fileobj(file_data, bucket_name, object_key)
    
    s3_path = f"s3://{bucket_name}/{object_key}"
    return s3_path


### Lang chain recursive character chunking
The most simplest way to chunk document would be by length, but keeping paragraphs or lines together so it does not lose the meaning. We will use langchain library's recursive character text splitter which offers ways to split data by length, yet keeps the lines, paragraph together as much as possible.


In [ ]:
# this method would split the text into chunks by paragraph, line boundary and keeping chunk 
# size as close to 1000 characters, it will also overlap the text between chunks if it were to 
# split line or paragraph in the middle.

def recursive_character_chunking(text): 
    
#     embeddings = BedrockEmbeddings() #create a Titan Embeddings client
    
#     pdf_path = "Amazon-com-Inc-2023-Annual-Report.pdf" #assumes local PDF file with this name

#     loader = PyPDFLoader(file_path=pdf_path) #load the pdf file
    
    text_splitter = RecursiveCharacterTextSplitter( #create a text splitter
        #separators=["\n\n", "\n", ".", " "], #split chunks at (1) paragraph, (2) line, (3) sentence, or (4) word, in that order
        chunk_size=1000, #divide into 1000-character chunks using the separators above
        chunk_overlap=100, #number of characters that can overlap with previous chunk
        length_function=len,
        is_separator_regex=False,
    )
    
    docs = text_splitter.create_documents(text)#From the loaded PDF
    
    return docs #return the index to be cached by the client app

Let's try to run this method on an excerpt of AWS docs from Amazon Bedrock titan model and Amazon Textract services. You will notice that length/recursive chunking will create chunks with overlaps, this helps in situations where sentences need to not be chopped in the middle, but it will fail to keep Textract and Titan documentation chunks separate. You will notice that chunk no. 9 is a mix of titan and lambda docs.

In [ ]:
text = ""

#lets load text from our prepared aws-docs-excerpt from various services.
with open('aws-docs-excerpt.txt', 'r') as f:
    text = f.read()


docs = recursive_character_chunking([text])

# the method prints chunks
def print_chunks(data):
    #Let's print the chunks -- notice the overlap between chunk 3 and 4
    i = 1
    for doc in data:
        print(f"---------START OF CHUNK {i}------")
        print(f"{doc.page_content}")
        print(f"---------END OF CHUNK {i}------\n\n")
        i+=1
        
print_chunks(docs)

## PDF Parsing
We have to convert our files into a String object before we can perform any form of chunking strategy. Here, we will parse our PDF into a string using PyPDFLoader from Langchain. This library will attempt to retain the format of the texts as much as possible.

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Amazon-com-Inc-2023-Annual-Report.pdf")
documents = loader.load()

#print(documents)

texts = ""

for document in documents:
    texts += document.page_content

### Semantic chunking
Semantic chunking is a novel technique that chunks the data in a way that it optimises it for semantic cohesion. The method uses an embedding model and runs similarity calculation over sentences and decides the chunk position based on deviation/change in semantic distance between sentences. It uses rolling window where it keeps adding sentences and measure its distance with incoming sentence. Technically a change in topic should be detected (not very accurately). A breakpoint threshold is statistical method use to determine this change. This way you ensure that chunks stay optimal for semantic matching. 

If you are keen to get more info, read level 4 in [Greg Kamradt tutorial](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb).

Langchain offers semantic chunking and also ability to call embedding model. We will first choose an embedding model for our semantic chunking a a breakpoint threshold type. After selecting the model and threshold, please move to the next cell.

In [ ]:
#lets initialize the code for drop down box input.
import ipywidgets as widgets
from ipywidgets import interactive

#defaults
model_id='amazon.titan-embed-text-v2:0'
threshold = 'percentile'

#list of embedding models in bedrock
model_list=['cohere.embed-english-v3','cohere.embed-multilingual-v3',
            'amazon.titan-embed-text-v1','amazon.titan-embed-text-v2:0',
           'amazon.titan-embed-image-v1']

#semantic chunking 
threshold_list=['percentile', 'standard_deviation', 'interquartile']
    
drop1 = widgets.Dropdown(options=model_list, value='cohere.embed-english-v3', description='Model:', disabled=False)
drop2 = widgets.Dropdown(options=threshold_list, value='percentile', description='Threshold:', disabled=False)

def get_model_dimension(model_id):
    if model_id=="amazon.titan-embed-text-v2:0":
        return 1024
    if model_id.startswith("cohere"):
        return 512
    if model_id.startswith("amazon.titan-embed-text-v1"):
        return 8192
    if model_id.startswith("amazon.titan-embed-image-v1"):
        return 8192

Following code runs semantic chunking for aws docs text. It also shows a drop down for you to change the model and threshold type so you can see the effects of various models and breakpoint thresholds

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
#from langchain_community.embeddings import BedrockEmbedding
from langchain_community.document_loaders import PDFMinerLoader

#method that is called when drop down boxes are shown or changed
def update_dropdown(selected_model, selected_threshold):
    model_id = selected_model.lower()
    threshold = selected_threshold.lower()
    info = f"Selected embedding model: {model_id}. Selected threshold: {threshold}!"
    display(info)
    semantic_chunks = perform_semantic_chunking(text=text, model_id=model_id, threshold=threshold)
    print_chunks(semantic_chunks)

    
# method runs semantic chunking on text for a given model and threshold.    
def perform_semantic_chunking(text, model_id, threshold):
    print(f"Chunking using {model_id} and {threshold} threshold breaking point")
    
    #using lang chain's Bedrock embedding object
    embeddings = BedrockEmbeddings(region_name=region, model_id=model_id)

    #using lang chain's semantic chunker to chunk
    text_splitter = SemanticChunker(
        embeddings, breakpoint_threshold_type= threshold
    )

    docs = text_splitter.create_documents([text])
    return docs

#lets run semantic chunking and display the drop down. 
w = interactive(update_dropdown, selected_model=drop1, selected_threshold=drop2) 
display(w)

#when you change value - give it takes a 10-15 seconds for refreshing the chunks

You can run above cell a number of with different combination of the embedding model and threshold ids to see what breaks the content best. You will find results vary from one model to another and between various breakpoint threshold technique. However, it does not mean this combination will always be best for chunking. Note that this also does not mean it is optimal for retrieval. We will have to test this with our queries to know if this is best to answer our questions.

### Loading text chunks in opensearch to run semantic search over chunks

#### Let's first create an index with KNN field.


In [ ]:
knn_index = {
    "settings": {
        "index.knn": True,
        "index.knn.space_type": "cosinesimil",
        "analysis": {
          "analyzer": {
            "default": {
              "type": "standard",
              "stopwords": "_english_"
            }
          }
        }
    },
    "mappings": {
        "properties": {
            "chunk_vector": {
                "type": "knn_vector",
                "dimension": get_model_dimension(model_id=model_id),
                "store": True
            },
            "chunk_content": {
                "type": "text",
                "store": True
            }
        }
    }
}


### Connect to Amazon OpenSearch Service

In [ ]:
kms = boto3.client('secretsmanager')
aos_credentials = json.loads(kms.get_secret_value(SecretId=outputs['OpenSearchSecret'])['SecretString'])

# For this lab we will use credentials that we have already created in AWS Secrets manager service. Secrets
# manager service allows you to store secrets securily and retrieve it through code in a safe manner.

auth = (aos_credentials['username'], aos_credentials['password'])

aos_client = OpenSearch(
    hosts = [{'host': aos_host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

In [ ]:
index_name = "aws_docs_index"

try:
    aos_client.indices.delete(index=index_name)
    print("Recreating index '" + index_name + "' on cluster.")
    aos_client.indices.create(index=index_name,body=knn_index,ignore=400)
except:
    print("Index '" + index_name + "' not found. Creating index on cluster.")
    aos_client.indices.create(index=index_name,body=knn_index,ignore=400)


In [ ]:
from utilities import embed_phrase, opensearch_bulk_load

#test calling embed_phrase method from utilities file to get embedding of a given model.
embed_phrase("Testing amazon bedrock models", model_id, bedrock_client=bedrock_client)

### Loading data in to opensearch
Let's load data into opensearch

In [ ]:
#process all the chunks and get embeddings from Bedrock for each text chunk
chunks = []

for doc in docs:
    chunks.append({"chunk_content": doc.page_content, "chunk_vector": embed_phrase(doc.page_content, model_id, bedrock_client)})

print(chunks[0])

#load data into opensearch - every chunk will be separate opensearch record/document.
opensearch_bulk_load(chunks, index_name, aos_client)

In [ ]:
def retrieve_opensearch_with_semantic_search(phrase, model_id, bedrock_client, n=3 ):
    search_vector = embed_phrase(phrase, model_id=model_id, bedrock_client=bedrock_client)
    osquery={
        "_source": {
            "exclude": [ "chunk_vector" ]
        },
        
      "size": n,
      "query": {
        "knn": {
          "chunk_vector": {
            "vector":search_vector,
            "k":n
          }
        }
      }
    }

    res = aos_client.search(index=index_name, 
                           body=osquery,
                           stored_fields=["chunk_content"],
                           explain = True)
    top_result = res['hits']['hits']
    
    results = []
    
    for entry in top_result:
        result = {
            "chunk_content":entry['_source']['chunk_content'],
        }
        results.append(result)
    
    return results



In [ ]:
question_on_s3_docs="What VPN connections do ?"

example_request = retrieve_opensearch_with_semantic_search(phrase=question_on_s3_docs, model_id=model_id, bedrock_client=bedrock_client, n=2)
print(json.dumps(example_request, indent=4))

## Building a RAG pipeline with Langchain
Work in Progress

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context" : retrieve_opensearch_with_semantic_search, "question" : RunnablePassthrough()}
    | rag_prompt
    | chat_model
    | StrOutputParser()
)